# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [6]:
import pandas as pd
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [51]:
document = ET.parse( './data/mondial_database.xml' )

In [58]:
# 1. 10 countries with the lowest infant mortality rates

# convert xml to pandas dataframe for easier processing
def parse_countries_to_df(countries):
    for country in countries:
        country_dict = {}
        country_dict['name'] = country.find('name').text
        if country.find('infant_mortality') is not None:
            country_dict['infant_mortality'] = country.find('infant_mortality').text
        
        yield country_dict
        
countries = document_root.findall('country')
countries_df = pd.DataFrame(list(parse_countries_to_df(countries)))

countries_df

,infant_mortality,name
0,13.19,Albania
1,4.78,Greece
2,7.9,Macedonia
3,6.16,Serbia
4,NaN,Montenegro
5,NaN,Kosovo
6,3.69,Andorra
7,3.31,France
8,3.33,Spain
9,4.16,Austria


In [69]:
# convert infant_mortality column to floats
countries_df['infant_mortality'] = countries_df['infant_mortality'].apply(pd.to_numeric)

# sort
sorted_countries = countries_df.sort_values('infant_mortality')

# result
sorted_countries[['name','infant_mortality']].head(10)

,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [120]:
# 10 cities with the largest population

def get_item_latest_population(item): # item can be either city or country
    populations = item.findall('population')
    populationsCount = len(populations) 
        
    if populationsCount > 0:
        return populations[populationsCount - 1].text
    else:
        return 0
            
def parse_cities_to_df(cities):
    for city in cities:
        city_dict = {}
        city_dict['name'] = city.find('name').text
        city_dict['population'] = get_item_latest_population(city)
        yield city_dict

cities = document_root.findall('country/city')
cities_df = pd.DataFrame(list(parse_cities_to_df(cities)))

cities_df

,name,population
0,Tirana,418495
1,Shkodër,77075
2,Durrës,113249
3,Vlorë,79513
4,Elbasan,78703
5,Korçë,51152
6,Skopje,514967
7,Kumanovo,107745
8,Beograd,1639121
9,Novi Sad,335701


In [121]:
# convert population column to floats
cities_df['population'] = cities_df['population'].apply(pd.to_numeric)

# sort
sorted_cities = cities_df.sort_values('population', ascending=False)

# result
sorted_cities[['name','population']].head(10)

,name,population
176,Seoul,9708483
164,Al Qahirah,8471859
80,Bangkok,7506700
128,Hong Kong,7055071
92,Ho Chi Minh,5968384
212,Singapore,5076700
163,Al Iskandariyah,4123869
216,New Taipei,3939305
177,Busan,3403135
107,Pyongyang,3255288


In [ ]:
# 3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [175]:
def get_group_percentage(group):
    return float(group.attrib['percentage'])

def get_group_population(group, country_population):
    return (get_group_percentage(group) / 100) * country_population
    
def create_group(group, country):
    country_population = float(get_latest_population(country))
    
    group_dict = {}
    group_dict['group'] = group.text
    group_dict['country'] = country.find('name').text
    group_dict['population'] = get_group_population(group, country_population)
    return group_dict
    
def get_country_groups(country):
    groups_list = []
    
    groups = country.findall('ethnicgroup')
    
    if len(groups) > 0:
        for group in groups:
            groups_list.append(create_group(group, country))
    
    return groups_list

def get_countries_groups(countries):
    groups = []
    for country in countries:
        groups += get_country_groups(country)
    
    return groups
    
countries = document_root.findall('country')
groups_df = pd.DataFrame(list(get_countries_groups(countries)))
groups_df

,country,group,population
0,Albania,Albanian,2.660131e+06
1,Albania,Greek,8.400414e+04
2,Greece,Greek,1.005915e+07
3,Macedonia,Macedonian,1.322388e+06
4,Macedonia,Albanian,5.190681e+05
5,Macedonia,Turkish,8.033197e+04
6,Macedonia,Gypsy,5.561444e+04
7,Macedonia,Serb,3.707629e+04
8,Serbia,Serb,5.903032e+06
9,Serbia,Montenegrin,6.408599e+04


In [181]:
# sum up the same ethnic groups in different countries
group_totals = groups_df.groupby('group').sum()

# sort 
sorted_group_totals = group_totals.sort_values('population', ascending=False)

# result
sorted_group_totals.head(10)

,population
group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08
